<a href="https://colab.research.google.com/github/kushalshah0/colab_tools/blob/main/idToJson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **DB Downloader**
import requests
from bs4 import BeautifulSoup
import json
from collections import OrderedDict
import time

def fetch_user_details(user_id):
    url = ""
    params = {"u": user_id}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        image_url = soup.select_one("#ContentPlaceHolder1_studentImage img")['src']
        rows = soup.select("table.table-striped.table-bordered tbody tr")
        data = OrderedDict()
        data["Image"] = image_url
        for row in rows:
            key = row.find_all('td')[0].text.strip()
            value = row.find_all('td')[1].text.strip()
            data[key] = value
        return data
    else:
        print(f"Failed to retrieve data for user ID {user_id}: {response.status_code}")
        return None

start_user_id =
end_user_id =

all_user_details = []

for user_id in range(start_user_id, end_user_id + 1):
    user_details = fetch_user_details(user_id)
    if user_details:
        all_user_details.append(user_details)
    time.sleep(1)

json_data = json.dumps(all_user_details, indent=4)

with open("user_details.json", "w") as json_file:
    json_file.write(json_data)

print("User details have been successfully fetched and saved to user_details.json")

In [ ]:
#@title **images Downloader**
import json
import requests
import os

def download_images_from_json(json_file):
    # Load the JSON data
    with open(json_file, 'r') as file:
        data = json.load(file)

    # Create an images directory if it doesn't exist
    if not os.path.exists('images'):
        os.makedirs('images')

    # Iterate through each entry in the JSON data
    for entry in data:
        image_url = entry.get("Image")
        roll_no = entry.get("RollNo")

        if image_url:
            # Download the image
            try:
                response = requests.get(image_url)
                response.raise_for_status()  # Raise an error for bad responses
                # Save the image with RollNo as the filename
                image_path = f'images/{roll_no}.jpg'
                with open(image_path, 'wb') as img_file:
                    img_file.write(response.content)
                print(f"Downloaded and saved: {image_path}")
            except requests.exceptions.RequestException as e:
                print(f"Failed to download the image for {roll_no}.jpg: {e}")
        else:
            print(f"Missing image URL for RollNo: {roll_no}. Skipping this entry.")

# Specify the path to your JSON file
json_file_path = '/content/user_details.json'  # Replace with your actual JSON file path
download_images_from_json(json_file_path)

In [ ]:
#@title **/images Compression**
import os
import zipfile

def zip_images_directory(directory):
    zip_filename = 'images.zip'

    # Create a zip file with no compression
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_STORED) as zip_file:
        # Walk through the directory
        for foldername, subfolders, filenames in os.walk(directory):
            for filename in filenames:
                file_path = os.path.join(foldername, filename)
                # Add the file to the zip file, maintaining the directory structure
                zip_file.write(file_path, os.path.relpath(file_path, directory))

    print(f"Zipped the images directory into: {zip_filename}")

# Specify the images directory to zip
images_directory = 'images'  # Make sure this directory exists
zip_images_directory(images_directory)